# Silver-to-Gold Agent Flow

## Overview
This notebook demonstrates the **data mart creation process** of the Agentic ELT Pipeline.
The Gold Layer transforms cleaned Silver data into business-ready analytics tables.

### Agent Workflow:
1. **Gold Draft Agent** → Analyzes Silver data and designs star schema
2. **Gold Builder Agent** → Generates data mart code
3. **Gold Runner** → Creates dimensions, facts, and aggregate tables

### What you'll learn:
- How LLM agents design star schemas for analytics
- Which business metrics and KPIs are automatically identified
- How agents create optimized data marts for BI tools

## 📋 Setup and Configuration

**What happens here:**
- Load required Python libraries
- Define paths to Silver, Gold, and draft report artifacts
- Verify data availability across different pipeline stages

In [ ]:
import json
import pandas as pd
from pathlib import Path
import yaml

# Set your Run ID here
run_id = "20260124_231016_#6470e523"

# Define paths
silver_dir = Path(f"../artifacts/silver/{run_id}")
gold_dir = Path(f"../artifacts/gold/marts/{run_id}")
draft_reports_dir = Path(f"../tmp/draft_reports/gold/{run_id}")

print(f"🔍 Checking data directories:")
print(f"Silver Directory: {silver_dir} ({'✅ exists' if silver_dir.exists() else '❌ missing'})")
print(f"Gold Directory: {gold_dir} ({'✅ exists' if gold_dir.exists() else '❌ missing'})")
print(f"Draft Reports: {draft_reports_dir} ({'✅ exists' if draft_reports_dir.exists() else '❌ missing'})")

## 🥈 Silver Layer Analysis

**What happens here:**
- Analyze cleaned Silver data that serves as input for Gold layer design
- Review Silver agent context and performance metrics
- Categorize tables by business domain (customers, products, sales, locations)

**Why important:**
The Gold Draft Agent uses this Silver layer analysis to understand available data structures and design appropriate star schema dimensions and facts.

In [ ]:
# Analyze Silver Agent Context
silver_context_path = silver_dir / "reports" / "silver_run_agent_context.json"
if silver_context_path.exists():
    with open(silver_context_path, 'r') as f:
        silver_context = json.load(f)
    
    print("📊 Silver Context for Gold Planning:")
    print(f"Run ID: {silver_context.get('run_id')}")
    print(f"Files processed: {silver_context.get('files_total')}")
    
    # Performance metrics
    perf = silver_context.get('performance_metrics', {})
    print(f"\n📈 Silver Layer Performance:")
    print(f"  Total rows: {perf.get('total_rows_processed', 0):,}")
    print(f"  Total columns: {perf.get('total_columns_processed', 0):,}")
    print(f"  Tables processed: {perf.get('tables_processed', 0)}")
    print(f"  Tables failed: {perf.get('tables_failed', 0)}")
else:
    print("❌ Silver Agent Context not found")

In [ ]:
# Categorize Silver tables for star schema design
silver_data_dir = silver_dir / "data"
if silver_data_dir.exists():
    csv_files = list(silver_data_dir.glob("*.csv"))
    
    print(f"🗂️ Silver Tables for Gold Design ({len(csv_files)} tables):")
    
    # Categorize tables by business domain
    customer_tables = []
    product_tables = []
    sales_tables = []
    location_tables = []
    other_tables = []
    
    for csv_file in csv_files:
        name = csv_file.name.lower()
        if 'cst' in name or 'customer' in name:
            customer_tables.append(csv_file.name)
        elif 'prd' in name or 'product' in name:
            product_tables.append(csv_file.name)
        elif 'sales' in name:
            sales_tables.append(csv_file.name)
        elif 'loc' in name or 'location' in name:
            location_tables.append(csv_file.name)
        else:
            other_tables.append(csv_file.name)
    
    print(f"\n🏗️ Star Schema Design Categories:")
    print(f"  Customer Dimension Sources: {customer_tables}")
    print(f"  Product Dimension Sources: {product_tables}")
    print(f"  Location Dimension Sources: {location_tables}")
    print(f"  Sales Fact Sources: {sales_tables}")
    print(f"  Other Tables: {other_tables}")
else:
    print("❌ Silver data not found")

## 🤖 Gold Draft Agent - Star Schema Design

**What happens here:**
- Review the LLM-generated star schema design from the Gold Draft Agent
- Analyze proposed dimensions, facts, and business metrics
- Understand the agent's reasoning for data mart architecture

**Agent Logic:**
The Gold Draft Agent analyzes Silver data to:
- Design dimension tables for master data (customers, products, locations)
- Create fact tables for transactional data (sales, orders)
- Identify aggregation opportunities for performance
- Plan wide tables for ad-hoc analytics

In [ ]:
# Analyze Gold Design Report
gold_design_path = draft_reports_dir / "gold_run_human_report.md"
if gold_design_path.exists():
    with open(gold_design_path, 'r', encoding='utf-8') as f:
        gold_design = f.read()
    
    print("📄 Gold Draft Agent - Design Report:")
    print("=" * 60)
    
    # Show first part of design report
    lines = gold_design.split('\n')
    for i, line in enumerate(lines[:35]):  # First 35 lines
        print(line)
    
    if len(lines) > 35:
        print(f"\n... ({len(lines)-35} more lines)")
    
    print(f"\n📊 Design Report Statistics:")
    print(f"  Total length: {len(gold_design):,} characters")
    print(f"  Lines: {len(lines)}")
    print(f"  Sections: {gold_design.count('#')} (estimated)")
else:
    print("❌ Gold Design Report not found")

## 📋 Gold Mart Plan - LLM Architecture

**What happens here:**
- Inspect the structured JSON mart plan created by the Gold Draft Agent
- Review planned dimensions, facts, and data mart specifications
- Understand the agent's prioritization and risk assessment

**Mart Plan Structure:**
The Gold Draft Agent creates a detailed plan that the Gold Builder Agent uses to generate executable code for data mart creation.

In [ ]:
# Analyze Gold Agent Context (Mart Plan)
gold_context_path = draft_reports_dir / "gold_run_agent_context.json"
if gold_context_path.exists():
    with open(gold_context_path, 'r') as f:
        gold_context = json.load(f)
    
    print("🤖 Gold Draft Agent - Mart Plan:")
    print(f"Objective: {gold_context.get('gold_layer_objective', 'N/A')}")
    
    # Dimensions analysis
    dimensions = gold_context.get('dimensions', [])
    print(f"\n🏛️ Planned Dimensions ({len(dimensions)}):")
    for dim in dimensions:
        print(f"  📊 {dim.get('name')}: {dim.get('grain')}")
        print(f"    Keys: {dim.get('primary_keys')}")
        attrs = dim.get('attributes', [])
        print(f"    Attributes: {attrs[:3]}{'...' if len(attrs) > 3 else ''}")
        print(f"    SCD: {dim.get('scd_guidance', 'N/A')}")
    
    # Facts analysis
    facts = gold_context.get('facts', [])
    print(f"\n📈 Planned Facts ({len(facts)}):")
    for fact in facts:
        print(f"  📊 {fact.get('name')}: {fact.get('grain')}")
        print(f"    Keys: {fact.get('primary_keys')}")
        measures = fact.get('measures', [])
        print(f"    Measures: {measures[:3]}{'...' if len(measures) > 3 else ''}")
        agg_ops = fact.get('aggregation_opportunities', [])
        print(f"    Aggregations: {len(agg_ops)} opportunities")
    
    # Marts summary
    marts = gold_context.get('marts', [])
    mandatory_marts = [m for m in marts if m.get('priority') == 'mandatory']
    optional_marts = [m for m in marts if m.get('priority') == 'optional']
    
    print(f"\n🏪 Planned Data Marts ({len(marts)} total):")
    print(f"  Mandatory: {len(mandatory_marts)} marts")
    print(f"  Optional: {len(optional_marts)} marts")
    
    for mart in mandatory_marts:
        print(f"    ✅ {mart.get('name')} ({mart.get('type')})")
    
    # Risk assessment
    risks = gold_context.get('risks', [])
    next_steps = gold_context.get('next_steps', [])
    print(f"\n⚠️ Risk Assessment: {len(risks)} risks identified")
    print(f"📋 Implementation Plan: {len(next_steps)} steps defined")
else:
    print("❌ Gold Agent Context not found")

## 🥇 Gold Data Marts Analysis

**What happens here:**
- Inspect the created Gold data marts and their structure
- Categorize tables by type (dimensions, facts, aggregates, wide tables)
- Analyze the implemented star schema and data relationships

**Data Mart Types:**
The Gold Runner creates different types of tables optimized for various analytics use cases.

In [ ]:
# Analyze Gold data marts
gold_data_dir = gold_dir / "data"
if gold_data_dir.exists():
    gold_csv_files = list(gold_data_dir.glob("*.csv"))
    
    print(f"🥇 Gold Data Marts ({len(gold_csv_files)} tables):")
    
    # Categorize Gold tables by type
    dimensions = []
    facts = []
    aggregates = []
    wide_tables = []
    other_tables = []
    
    for csv_file in gold_csv_files:
        name = csv_file.name.lower()
        if 'dim_' in name:
            dimensions.append(csv_file.name)
        elif 'fact_' in name:
            facts.append(csv_file.name)
        elif 'agg_' in name:
            aggregates.append(csv_file.name)
        elif 'wide_' in name:
            wide_tables.append(csv_file.name)
        else:
            other_tables.append(csv_file.name)
    
    print(f"\n🏗️ Star Schema Implementation:")
    print(f"  🏛️ Dimensions: {dimensions}")
    print(f"  📈 Facts: {facts}")
    print(f"  📊 Aggregates: {aggregates}")
    print(f"  📋 Wide Tables: {wide_tables}")
    print(f"  ❓ Other: {other_tables}")
    
    # Detailed table analysis
    print(f"\n📊 Detailed Table Analysis:")
    for csv_file in gold_csv_files:
        df = pd.read_csv(csv_file)
        table_type = "📊 Other"
        if csv_file.name in dimensions:
            table_type = "🏛️ Dimension"
        elif csv_file.name in facts:
            table_type = "📈 Fact"
        elif csv_file.name in aggregates:
            table_type = "📊 Aggregate"
        elif csv_file.name in wide_tables:
            table_type = "📋 Wide Table"
        
        print(f"\n  {table_type} {csv_file.name}:")
        print(f"    Dimensions: {df.shape[0]:,} rows × {df.shape[1]} columns")
        print(f"    Columns: {list(df.columns)}")
        
        if len(df) > 0:
            # Show sample data structure
            sample_row = df.iloc[0].to_dict()
            print(f"    Sample: {list(sample_row.keys())[:5]}{'...' if len(sample_row) > 5 else ''}")
else:
    print("❌ Gold data not found")

## 📈 Business KPIs and Metrics

**What happens here:**
- Analyze available business metrics in the Gold layer
- Calculate sample KPIs from fact tables
- Identify dimensional analysis opportunities

**KPI Analysis:**
Demonstrate how the Gold layer enables business intelligence and analytics by providing ready-to-use metrics.

In [ ]:
# Analyze business KPIs from fact tables
if gold_data_dir.exists():
    fact_files = [f for f in gold_csv_files if 'fact_' in f.name.lower()]
    
    print(f"📈 BUSINESS KPI ANALYSIS")
    print("=" * 50)
    
    for fact_file in fact_files:
        df_fact = pd.read_csv(gold_data_dir / fact_file.name)
        print(f"\n📊 KPI Analysis for {fact_file.name}:")
        
        # Identify numeric columns for metrics
        numeric_cols = df_fact.select_dtypes(include=['number']).columns.tolist()
        print(f"  Available Metrics: {numeric_cols}")
        
        # Calculate sample KPIs
        if len(numeric_cols) > 0:
            for col in numeric_cols[:3]:  # First 3 numeric columns
                if df_fact[col].notna().any():
                    print(f"\n  📊 {col} Statistics:")
                    print(f"    Total: {df_fact[col].sum():,.2f}")
                    print(f"    Average: {df_fact[col].mean():,.2f}")
                    print(f"    Min: {df_fact[col].min():,.2f}")
                    print(f"    Max: {df_fact[col].max():,.2f}")
        
        # Dimensional analysis opportunities
        date_cols = [col for col in df_fact.columns if 'date' in col.lower() or 'time' in col.lower()]
        id_cols = [col for col in df_fact.columns if 'id' in col.lower() and col.lower() != 'id']
        
        if date_cols:
            print(f"\n  📅 Time Dimensions: {date_cols}")
        
        if id_cols:
            print(f"  🔗 Dimensional Keys: {id_cols}")
            for id_col in id_cols[:2]:  # First 2 ID columns
                unique_count = df_fact[id_col].nunique()
                print(f"    {id_col}: {unique_count:,} unique values")
else:
    print("❌ Cannot analyze KPIs - Gold data not available")

## ✅ Data Mart Quality Assessment

**What happens here:**
- Review Gold layer metadata and execution results
- Assess data mart creation success rates
- Validate star schema implementation quality

**Quality Metrics:**
Evaluate the success of the Gold layer transformation and data mart creation process.

In [ ]:
# Analyze Gold layer metadata
gold_metadata_path = gold_dir / "metadata.yaml"
if gold_metadata_path.exists():
    with open(gold_metadata_path, 'r') as f:
        gold_metadata = yaml.safe_load(f)
    
    print("📋 GOLD LAYER QUALITY ASSESSMENT")
    print("=" * 50)
    
    print(f"\n📊 Execution Summary:")
    print(f"  Run ID: {gold_metadata.get('run_id')}")
    print(f"  Layer: {gold_metadata.get('layer')}")
    print(f"  Source Layer: {gold_metadata.get('source_layer')}")
    
    summary = gold_metadata.get('summary', {})
    print(f"\n📈 Processing Results:")
    print(f"  Files total: {summary.get('files_total')}")
    print(f"  Files success: {summary.get('files_success')}")
    print(f"  Files failed: {summary.get('files_failed')}")
    
    # Calculate success rate
    total_files = summary.get('files_total', 0)
    success_files = summary.get('files_success', 0)
    if total_files > 0:
        success_rate = (success_files / total_files) * 100
        print(f"  Success Rate: {success_rate:.1f}%")
    
    # Table-level results
    tables = gold_metadata.get('tables', {})
    print(f"\n📊 Table-Level Results:")
    successful_tables = 0
    total_rows = 0
    
    for table_name, table_info in tables.items():
        status = table_info.get('status', 'UNKNOWN')
        rows = table_info.get('rows_out', 0)
        total_rows += rows
        
        status_icon = "✅" if status == "SUCCESS" else "❌"
        print(f"  {status_icon} {table_name}: {status} ({rows:,} rows)")
        
        if status == "SUCCESS":
            successful_tables += 1
    
    print(f"\n🎯 Quality Metrics:")
    print(f"  Successful Tables: {successful_tables}/{len(tables)}")
    print(f"  Total Rows Created: {total_rows:,}")
    print(f"  Average Rows per Table: {total_rows/len(tables) if tables else 0:,.0f}")
else:
    print("❌ Gold metadata not found")

## 🔄 Agent Workflow Summary

**What you learned:**
This section summarizes the entire Gold Agent workflow and explains how LLM agents create business-ready data marts from cleaned Silver data.

In [ ]:
print("🤖 GOLD AGENT WORKFLOW - SUMMARY")
print("=" * 60)
print()
print("1️⃣ GOLD DRAFT AGENT (Architecture & Design):")
print("   🔍 Reads Silver Agent Context and metadata")
print("   📊 Analyzes available tables and schemas")
print("   🏗️ Designs star schema (dimensions, facts, aggregates)")
print("   📈 Identifies business KPIs and metrics")
print("   📄 Creates gold_run_human_report.md (design rationale)")
print("   📋 Creates gold_run_agent_context.json (mart plan)")
print()
print("2️⃣ GOLD BUILDER AGENT (Code Generation):")
print("   📖 Reads Gold Draft Context")
print("   🛠️ Generates executable Python code for data marts")
print("   🏛️ Implements dimension tables (SCD Type 1/2)")
print("   📈 Implements fact tables with measures")
print("   📊 Implements aggregate tables for performance")
print("   📜 Creates load_3_gold_layer_runner.py")
print()
print("3️⃣ GOLD RUNNER (Execution):")
print("   ▶️ Executes generated code")
print("   🔄 Transforms Silver → Gold data marts")
print("   🏗️ Creates star schema tables")
print("   💾 Generates business-ready CSV files")
print("   📋 Creates metadata and reports")
print()
print("✅ RESULT: Business-ready data marts for BI/Analytics")
print("   🏛️ Dimensions for master data")
print("   📈 Facts for transactional metrics")
print("   📊 Aggregates for performance-optimized KPIs")
print("   📋 Wide tables for ad-hoc analysis")
print("   🔗 Optimized for BI tools (Tableau, Power BI)")
print("   📊 Ready-to-use business metrics and KPIs")